#### Indentify and handle missing values, if any.
#### Check for and resolve any inconsistencies or error in the data
#### Ensure that the data type of each column are appropriate

### Importing necessary libraries for data cleansing

In [2]:
import pandas as pd 
import numpy as np 
import re

In [3]:
dataset = pd.read_csv("car.csv")
df = dataset.copy()
df.head()

,Name,Price,Used For,Transmisson,Colour,Make Year,Mileage,Engine (CC),Fuel,Kilometer Run,Waranty,Types
0,Hyundai | i20 Active S | TDi | 2015 | Hatchbac...,"à¤°à¥‚. 24,75,000à¤°à¥‚. 25,00,000",Private Use,Manual2WD,Brown,2015,14,1400,Petrol,42000,NaN,NaN
1,Excellent car on sale (Hyundai),"à¤°à¥‚. 7,50,000",NaN,Auto2WD,Light blue,2005,11,1399,Petrol,87412,NaN,NaN
2,TATA 407 Container (Tata),"à¤°à¥‚. 7,00,000",NaN,Manual - 2WD,White,2013,NaN,2956,Diesel,60000,NaN,NaN
3,4x4 swaraj Mazda (Mahindra),"à¤°à¥‚. 6,00,000",NaN,Manual - 4WD,NaN,2017,NaN,NaN,Diesel,NaN,NaN,NaN
4,i20 Active good for used few time (Hyundai),à¤°à¥‚. 375,NaN,Auto - 2WD,white,2019,17,1200,Petrol,2400,NaN,NaN


In [4]:
# cleaning all the column names at beginning
# 1. Replace spaces and special characters with underscores
# 2. Convert to lowercase
# 3. Fix known typos
df.columns = df.columns.str.replace(' ', '_').str.replace(r'[()]', '', regex=True).str.lower() 
df.rename(columns={'transmisson': 'transmission', 'waranty': 'warranty'}, inplace=True)

print("Cleaned Column Names:")# displaying clean name of the column
print(df.columns)

df.head() # Display the head.

Cleaned Column Names:
Index(['name', 'price', 'used_for', 'transmission', 'colour', 'make_year',
       'mileage', 'engine_cc', 'fuel', 'kilometer_run', 'warranty', 'types'],
      dtype='object')


,name,price,used_for,transmission,colour,make_year,mileage,engine_cc,fuel,kilometer_run,warranty,types
0,Hyundai | i20 Active S | TDi | 2015 | Hatchbac...,"à¤°à¥‚. 24,75,000à¤°à¥‚. 25,00,000",Private Use,Manual2WD,Brown,2015,14,1400,Petrol,42000,NaN,NaN
1,Excellent car on sale (Hyundai),"à¤°à¥‚. 7,50,000",NaN,Auto2WD,Light blue,2005,11,1399,Petrol,87412,NaN,NaN
2,TATA 407 Container (Tata),"à¤°à¥‚. 7,00,000",NaN,Manual - 2WD,White,2013,NaN,2956,Diesel,60000,NaN,NaN
3,4x4 swaraj Mazda (Mahindra),"à¤°à¥‚. 6,00,000",NaN,Manual - 4WD,NaN,2017,NaN,NaN,Diesel,NaN,NaN,NaN
4,i20 Active good for used few time (Hyundai),à¤°à¥‚. 375,NaN,Auto - 2WD,white,2019,17,1200,Petrol,2400,NaN,NaN


In [5]:
df_duplicates = df[df.duplicated()]
duplicate_count = df.duplicated().sum()

df.drop_duplicates(inplace=True)



## Handling the missing values

In [6]:
missing_values = df.isnull().sum().sort_values(ascending = False)
missing_values #0 for missing values

types            847
warranty         638
used_for         490
mileage          389
kilometer_run    141
engine_cc        106
colour            81
name               0
transmission       0
price              0
make_year          0
fuel               0
dtype: int64

### Engineering Feature from the warranty column


In [ ]:

df['warranty'] = df['warranty'].astype(str).str.lower() # First, ensure the column is a string type to use string methods, filling NaNs with an empty string

positive_keywords = r'year|month|company|yes|dealer|available|warranty|chaa|uears|accident free' # Define keywords that imply a warranty exists like year', 'month', 'company', 'yes', 'dealer', 'available'

#new binary column It's 1 if a positive keyword is found, and the string isn't just 'no' or 'not'
df['Has_Waranty'] = 0 # Default to 0 (no warranty)
df.loc[
    df['warranty'].str.contains(positive_keywords, na=False) & 
    ~df['warranty'].str.contains(r'\bno\b|\bnot\b', na=False),
    'has_warranty'
] = 1



## Feature engineering for 'Used_For'

In [8]:
df['used_for'] = df['used_for'].astype(str).str.lower()
private_keywords = r'private|personal|home|office|prasnal|persona|New'
commercial_keywords = r'school|taxi|commercial|supply|transportation|delivery|goods|Hosiptal|years|Tourist|company|driving'

df['is_private_use'] = df['used_for'].str.contains(private_keywords, na=False).astype(int)
df['is_commercial_use'] = df['used_for'].str.contains(commercial_keywords, na=False).astype(int)


## Drop the original messy and irrelevant columns

In [9]:
df.drop(columns=['warranty', 'used_for', 'types'], inplace=True) #warranty and used_for were messy and types is unused/ irrelevant

### printing clean columns

In [10]:

print(df['has_warranty'].value_counts())

print(df['is_private_use'].value_counts())

print(df['is_commercial_use'].value_counts())


print(df.columns)

has_warranty
1.0    47
Name: count, dtype: int64
is_private_use
0    738
1    109
Name: count, dtype: int64
is_commercial_use
0    740
1    107
Name: count, dtype: int64
Index(['name', 'price', 'transmission', 'colour', 'make_year', 'mileage',
       'engine_cc', 'fuel', 'kilometer_run', 'Has_Waranty', 'has_warranty',
       'is_private_use', 'is_commercial_use'],
      dtype='object')


## Correct Structural and Formatting Errors 

### 1. cleaning the Name and creating a Brand column

In [11]:
print("Current column names:", df.columns)

df['brand'] = df['name'].str.extract(r'\((.*?)\)').fillna('Unknown')

df['name'] = df['name'].str.replace(r'\s*\(.*\)', '', regex=True).str.strip()

df['brand'] = df['brand'].str.replace('other chinese brands', 'other').str.replace('other brands', 'other')

print(df[['name', 'brand']].head())

Current column names: Index(['name', 'price', 'transmission', 'colour', 'make_year', 'mileage',
       'engine_cc', 'fuel', 'kilometer_run', 'Has_Waranty', 'has_warranty',
       'is_private_use', 'is_commercial_use'],
      dtype='object')
                                              name     brand
0  Hyundai | i20 Active S | TDi | 2015 | Hatchback   Hyundai
1                            Excellent car on sale   Hyundai
2                               TATA 407 Container      Tata
3                                 4x4 swaraj Mazda  Mahindra
4                i20 Active good for used few time   Hyundai


### 2. cleaning price column

In [ ]:

def clean_price(price):
    if isinstance(price, str):
        match = re.search(r'[\d,]+', price)#find the number in the string 
    
        if match:
            number_string = match.group(0)
            cleaned_string = number_string.replace(',', '')
            return pd.to_numeric(cleaned_string, errors='coerce')#converts the cleaned string to a numeric value
    return np.nan

# 'dataset' has the original data with the 'Price' (capital P) column.
df['price'] = dataset['Price']

# apply the function because df['price'] 
df['price_npr'] = df['price'].apply(clean_price)

# drop the old column  you've  new one
df.drop(columns=['price'], inplace=True) 

print(df['price_npr'].head())

0    2475000
1     750000
2     700000
3     600000
4        375
Name: price_npr, dtype: int64


### 3. Cleaning Transmission

In [13]:
df['transmission'] = df['transmission'].str.lower().str.replace('-', ' ').str.strip()

#  Create 'transmission_type'
df['transmission_type'] = np.nan
df.loc[df['transmission'].str.contains('manual', na=False), 'transmission_type'] = 'Manual'
df.loc[df['transmission'].str.contains('auto', na=False), 'transmission_type'] = 'Automatic'
df['transmission_type'].fillna('Unknown', inplace=True)


#Create 'drivetrain'
df['drivetrain'] = np.nan
df.loc[df['transmission'].str.contains('2wd', na=False), 'drivetrain'] = '2WD'
df.loc[df['transmission'].str.contains('4wd', na=False), 'drivetrain'] = '4WD'
df['drivetrain'].fillna('Unknown', inplace=True)


#Drop the old column
df.drop(columns=['transmission'], inplace=True)


#Validate the results 
print("\n--- New 'transmission_type' and 'drivetrain' columns ---")
print(df[['transmission_type', 'drivetrain']].head())

print("\n--- Value Counts to check our work ---")
print("Transmission Types:\n", df['transmission_type'].value_counts())
print("\nDrivetrain Types:\n", df['drivetrain'].value_counts())


--- New 'transmission_type' and 'drivetrain' columns ---
  transmission_type drivetrain
0            Manual        2WD
1         Automatic        2WD
2            Manual        2WD
3            Manual        4WD
4         Automatic        2WD

--- Value Counts to check our work ---
Transmission Types:
 transmission_type
Manual       623
Automatic    180
Unknown       44
Name: count, dtype: int64

Drivetrain Types:
 drivetrain
2WD        647
4WD        156
Unknown     44
Name: count, dtype: int64


C:\Users\DELL\AppData\Local\Temp\ipykernel_10472\1751426811.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Manual' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['transmission'].str.contains('manual', na=False), 'transmission_type'] = 'Manual'
C:\Users\DELL\AppData\Local\Temp\ipykernel_10472\1751426811.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['transmission_type'].fillna('Un

## cleaning colour

In [14]:
# ensuring the column is treated as a string, convert to lowercase, and strip whitespace.
# This handles casing issues and any leading/trailing spaces.
df['colour'] = df['colour'].astype(str).str.lower().str.strip()


# Removing all characters that are not letters or spaces. Splitting by space and take the first word. This turns "polar white" into "polar" and "red + black" into "red".
df['colour'] = df['colour'].str.replace(r'[^a-z\s]', '', regex=True).str.split().str[0]

# Consolidate Color Variations with a Mapping Dictionary 
# Now we map the simplified words to a standard color.
color_map = {
    # White variations
    'polar': 'white', 'off': 'white', 'dew': 'white', 'pearl': 'white', 'whitr': 'white',
    # Silver variations
    'artic': 'silver', 'silky': 'silver', 'stardust': 'silver', 'sleek': 'silver', 
    'silbhar': 'silver', 'selki': 'silver', 'sliver': 'silver',
    # Grey variations
    'gray': 'grey', 'granite': 'grey', 'dust': 'grey', 'space': 'grey', 'smoke': 'grey',
    # Red variations
    'exo': 'red', 'dark': 'red', 'passion': 'red', 'maroon': 'red', 'marun': 'red', 
    'mehroon': 'red', 'read': 'red',
    # Blue variations
    'nilo': 'blue', 'metallic': 'blue', 'navy': 'blue', 'asure': 'blue', 'sky': 'blue', 'light': 'blue',
    # Brown variations
    'earth': 'brown', 'chocolate': 'brown',
    # Golden variations
    'goldenã³': 'golden', # Handles the encoding error text
    # Other Typos/Variations
    'black': 'black', 'white': 'white', 'silver': 'silver', 'grey': 'grey', 'red': 'red',
    'blue': 'blue', 'brown': 'brown', 'green': 'green', 'orange': 'orange', 'yellow': 'yellow',
    'golden': 'golden', 'purple': 'purple', 'cream': 'cream', 'beige': 'beige'
}

df['colour'] = df['colour'].replace(color_map)

#  Handling the Missing Values and Unmapped Remainders
# The original NaNs are now the string 'nan'. Replace it with 'Unknown'.
df['colour'].replace('nan', 'Unknown', inplace=True)

# Anything that wasn't in our map might be a rare color or junk. Let's check counts.
# We can optionally group very rare colors into an 'Other' category.
value_counts = df['colour'].value_counts()
to_other = value_counts[value_counts <= 3].index # Find colors that appear 3 or fewer times
df.loc[df['colour'].isin(to_other), 'colour'] = 'Other'


# Final Validation 
print("Cleaned 'colour' column")
print(df['colour'].head())

print("\n Value Counts After Cleaning ")
print(df['colour'].value_counts())

C:\Users\DELL\AppData\Local\Temp\ipykernel_10472\3345714653.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['colour'].replace('nan', 'Unknown', inplace=True)


Cleaned 'colour' column
0      brown
1       blue
2      white
3    Unknown
4      white
Name: colour, dtype: object

 Value Counts After Cleaning 
colour
silver     196
white      181
grey        92
Unknown     81
blue        80
red         71
Other       60
black       30
brown       13
golden       9
orange       7
yellow       6
cream        6
green        6
marron       5
Name: count, dtype: int64


###  Cleaning Make_Year, Mileage, Engine_CC, Kilometer_Run

In [15]:

raw_cols_to_clean = {
    'make_year': 'Make Year',
    'mileage': 'Mileage',
    'engine_cc': 'Engine (CC)',
    'kilometer_run': 'Kilometer Run'
}
for clean_name, raw_name in raw_cols_to_clean.items():
    if clean_name not in df.columns and raw_name in dataset.columns:
        print(f"Restoring column '{clean_name}' from original dataset !")
        df[clean_name] = dataset[raw_name]


# Define a Reusable Cleaning Function
def extract_first_number(value):
    if isinstance(value, str):
        match = re.search(r'(\d+\.?\d*)', value)
        if match:
            return pd.to_numeric(match.group(0), errors='coerce')
    return pd.to_numeric(value, errors='coerce')

#  Apply the Function to Each Column
df['make_year_cleaned'] = df['make_year'].apply(extract_first_number)
df['mileage_kmpl_cleaned'] = df['mileage'].apply(extract_first_number)
df['engine_cc_cleaned'] = df['engine_cc'].apply(extract_first_number)
df['kilometer_run_cleaned'] = df['kilometer_run'].apply(extract_first_number)

#  Sanity Check and Outlier Handling
print("Statistics Before Outlier Handling ")
print(df[['make_year_cleaned', 'mileage_kmpl_cleaned', 'engine_cc_cleaned', 'kilometer_run_cleaned']].describe())

df.loc[df['make_year_cleaned'] == 207, 'make_year_cleaned'] = 2017
df.loc[df['make_year_cleaned'] > 2024, 'make_year_cleaned'] = np.nan
df.loc[df['kilometer_run_cleaned'] < 100, 'kilometer_run_cleaned'] = np.nan
df.loc[df['kilometer_run_cleaned'] > 500000, 'kilometer_run_cleaned'] = np.nan
df.loc[df['engine_cc_cleaned'] < 100, 'engine_cc_cleaned'] = np.nan

# Impute Missing Values
for col in ['make_year_cleaned', 'mileage_kmpl_cleaned', 'engine_cc_cleaned', 'kilometer_run_cleaned']:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)

# Finalize and Validate
df['make_year_cleaned'] = df['make_year_cleaned'].astype(int)
df['kilometer_run_cleaned'] = df['kilometer_run_cleaned'].astype(int)

df.drop(columns=['make_year', 'mileage', 'engine_cc', 'kilometer_run'], inplace=True)

df.rename(columns={
    'make_year_cleaned': 'make_year',
    'mileage_kmpl_cleaned': 'mileage_kmpl',
    'engine_cc_cleaned': 'engine_cc',
    'kilometer_run_cleaned': 'kilometer_run'
}, inplace=True)

# Final Validation 
print("\n Final Cleaned Numeric Columns Info ")
df[['make_year', 'mileage_kmpl', 'engine_cc', 'kilometer_run']].info()

print("\nStatistics After Cleaning and Imputation ")
print(df[['make_year', 'mileage_kmpl', 'engine_cc', 'kilometer_run']].describe())

Statistics Before Outlier Handling 
       make_year_cleaned  mileage_kmpl_cleaned  engine_cc_cleaned  \
count         847.000000            455.000000         741.000000   
mean         1999.976387            392.563824        1510.386775   
std           160.513484           4063.584184         893.674242   
min             0.000000              5.000000           0.000000   
25%          2010.000000             12.000000        1196.000000   
50%          2014.000000             15.000000        1200.000000   
75%          2017.000000             17.000000        1600.000000   
max          2069.000000          69000.000000       15000.000000   

       kilometer_run_cleaned  
count           7.050000e+02  
mean            5.906926e+04  
std             1.014570e+05  
min             0.000000e+00  
25%             3.200000e+04  
50%             4.900000e+04  
75%             6.600000e+04  
max             1.700000e+06  

 Final Cleaned Numeric Columns Info 
<class 'pandas.core.frame

C:\Users\DELL\AppData\Local\Temp\ipykernel_10472\962710218.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_val, inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_10472\962710218.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

In [16]:
df.head()

,name,colour,fuel,Has_Waranty,has_warranty,is_private_use,is_commercial_use,brand,price_npr,transmission_type,drivetrain,make_year,mileage_kmpl,engine_cc,kilometer_run
0,Hyundai | i20 Active S | TDi | 2015 | Hatchback,brown,Petrol,0,NaN,1,0,Hyundai,2475000,Manual,2WD,2015,14.0,1400.0,42000
1,Excellent car on sale,blue,Petrol,0,NaN,0,0,Hyundai,750000,Automatic,2WD,2005,11.0,1399.0,87412
2,TATA 407 Container,white,Diesel,0,NaN,0,0,Tata,700000,Manual,2WD,2013,15.0,2956.0,60000
3,4x4 swaraj Mazda,Unknown,Diesel,0,NaN,0,0,Mahindra,600000,Manual,4WD,2017,15.0,1200.0,50000
4,i20 Active good for used few time,white,Petrol,0,NaN,0,0,Hyundai,375,Automatic,2WD,2019,17.0,1200.0,2400


In [17]:
missing_values = df.isnull().sum().sort_values(ascending = False)
missing_values

has_warranty         800
colour                 4
name                   0
fuel                   0
Has_Waranty            0
is_private_use         0
is_commercial_use      0
brand                  0
price_npr              0
transmission_type      0
drivetrain             0
make_year              0
mileage_kmpl           0
engine_cc              0
kilometer_run          0
dtype: int64

df.drop(columns=['has_warranty'], inplace=True) 